In [1]:
# 덧셈만 하는 deeplearing ai 만들기
# 숫자값을 int가 아니라 str 문자 하나로 생각해서 제작

In [2]:
#필요한 module import

import os
import sys
sys.path.append(os.path.join(os.path.dirname(""), ".."))

import custom
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

In [3]:
# 데이터 불러오기 전에 원핫 인코딩 리스트, 숫자-라벨링 사전 두개 만들어서 원핫 인코딩 리스트는 신경망 안에 집어넣기
#[' ',1,2,3,4,5,6,7,8,9,0,+]

#원한 인코딩 리스트
embedding_tensor = torch.cat([torch.zeros(1,11), torch.eye(11)])
#torch.zeros : 파라미터 크기의 0텐서 생성
#torch.eye : 정사각 단위행렬 (대각선이 1이고 나머진 0인 행렬) 생성
print(embedding_tensor)
#0번째 : 공백(padding), 1~10번째 : 1~0, 11번째 : +

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])


In [4]:
#숫자 문자 - 라벨링 사전

num_dict = {}

num_dict[" "] = 0
for i in range(1,10) :
    num_dict[str(i)] = i
num_dict["0"] = 10
num_dict["+"] = 11

print(num_dict)

{' ': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '0': 10, '+': 11}


In [5]:
# 데이터 읽어오기

with open("addition.txt", mode = "r") as f:
    ori_data = f.read()

In [6]:
# 데이터 x, t로 나누기
data = ori_data.split("\n")

x = []
t = []
for nums in data :
    if nums.find("_") < 0 :
        continue
    nums = nums.split("_") # _를 기준으로 나누고
    x.append(nums[0]) #앞의 입력값
    t.append(nums[1]) #뒤의 출력값

print("x의 갯수 : ", len(x))
print("x문자의 갯수 : ", len(x[10]))
print("x를 문자로 나눈 것 : ", list(x[10]))

print("t의 갯수 : ", len(t))
print("t문자의 갯수 : ", len(t[10]))
print("t를 문자로 나눈 것 : ", list(t[10]))

x의 갯수 :  50000
x문자의 갯수 :  7
x를 문자로 나눈 것 :  ['1', '8', '+', '8', ' ', ' ', ' ']
t의 갯수 :  50000
t문자의 갯수 :  4
t를 문자로 나눈 것 :  ['2', '6', ' ', ' ']


In [7]:
#x 문자들을 라벨링
vector_x = []
for i in range(len(x)) :
    temp = list(x[i]) #str을 문자단위로 끊은 list로 바꾸기
    vector = custom.word_vectorize(temp, num_dict)
    vector_x.append(vector)

vector_x = np.array(vector_x)
print("x 모양 : ", vector_x.shape)
print("10번째 라벨값 : ", vector_x[10])

x 모양 :  (50000, 7)
10번째 라벨값 :  [ 1  8 11  8  0  0  0]


In [8]:
#t 문자들을 라벨링 (출력값 y와 비교하기 위해서) 
vector_t = []
for i in range(len(t)) :
    temp = list(t[i])
    vector = custom.word_vectorize(temp, num_dict)
    vector_t.append(vector)

vector_t = np.array(vector_t)
print("t 모양 : ", vector_t.shape)
print("10번째 라벨값 : ", vector_t[10])

t 모양 :  (50000, 4)
10번째 라벨값 :  [2 6 0 0]


In [9]:
# 전처리한 데이터 x, t를 DataLoader에 싣기
# train, test 나누겠습니다. 앞의 40000개는 train, 뒤의 10000개는 test
device = "cuda" if torch.cuda.is_available() else "cpu"
s = 40000

tensor_x = torch.tensor(vector_x, dtype = torch.long, device = device)
tensor_t = torch.tensor(vector_t, dtype = torch.long, device = device)

train_zip_list = list(zip(tensor_x[:s], tensor_t[:s]))
test_zip_list = list(zip(tensor_x[s:], tensor_t[s:]))

train_dataloader = DataLoader(train_zip_list, batch_size=100, shuffle=True)
test_dataloader = DataLoader(test_zip_list, batch_size=1000, shuffle=False)

print(len(train_dataloader))
print(len(test_dataloader))

400
10


In [10]:
#AI 만들기
# x,t 입력은 데이터로더에서 완료

# 함수들 만들기
class Encoder(nn.Module) :
    def __init__(self, embedding_vector) :
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_vector, freeze = True, padding_idx = 0) # 라벨값을 벡터로 바꾸는 Embedding 함수
        self.rnn = nn.LSTM(embedding_vector.shape[1], embedding_vector.shape[1], batch_first = True, bidirectional = True) #임베딩 열 크기를 받아서 임베딩 열 크기를 출력
    def forward(self, x) :
        x = self.embedding(x)
        y, hc = self.rnn(x)
        return y, hc

class Decoder(nn.Module) :
    def __init__(self, embedding_vector) :
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_vector, freeze = True, padding_idx = 0)
        self.rnn = nn.LSTM(embedding_vector.shape[1] * 3, embedding_vector.shape[1], batch_first = True, bidirectional = True)
        self.f = nn.Linear(embedding_vector.shape[1] * 4, embedding_vector.shape[0]) #RNN을 계산하고 난 뒤 벡터값을 다시 라벨값으로 바꾸는 용도, 단어 출력 용도
        self.encoder_h_context = None #encoder_h 가공값 저장할 변수
    def forward(self, encoder_output, encoder_hc, t = None) : #문장 여러개 처리하기 위해서 encoder_output.shape가 필요함
        #decoder 학습 방법 2가지 (greedy, teaching_force)
        #encoder_h 가공
        encoder_h_forward = encoder_hc[0][0:1,:,:]
        encoder_h_backward = encoder_hc[0][1:2,:,:] #encoder의 h값 반으로 쪼개기
        self.encoder_h_context = torch.concat([encoder_h_forward, encoder_h_backward], dim = -1).transpose(0,1) #peeky algorithm, 적용하기 위한 변수
        
        batch_size = encoder_output.shape[0] #문장 갯수 정하기 = encoder에서 처리한 문장 갯수
        decoder_input = torch.zeros(batch_size, 1).type(torch.long).to(encoder_output.device) #처음 넣는 단어 (0번째 단어인 padding)
        decoder_hc = encoder_hc
        decoder_output_list = [] #단어들 모아서 문장으로 만들어 출력

        for i in range(4) : #t의 문자갯수만큼 반복
            decoder_output, decoder_hc = self.forward_cal(decoder_input, decoder_hc)
            decoder_output_list.append(decoder_output)

            if t is None : #greedy (t가 없음)
                decoder_input = decoder_output.argmax(dim = -1).detach()
            else : #teaching-force (t가 있음)
                decoder_input = t[:, i:i+1]

        decoder_output_list = torch.cat(decoder_output_list, dim = 1) #for 문 종료 후 list로 모아놓은 tensor들을 tensor로 합치는 작업
        return decoder_output_list, decoder_hc, None
        
    def forward_cal(self, x, h) : #실제 신경망 계산 함수
        x = self.embedding(x) #라벨을 벡터로
        x = torch.concat([self.encoder_h_context, x], dim = -1) #peeky algorithm 적용
        output, h = self.rnn(x, h) #처음 h값을 지정하고 싶으면 x 뒤에 h값 입력하세요
        output = torch.concat([self.encoder_h_context, output], dim = -1) #peeky algorithm 적용
        output = self.f(output) #벡터 계산값을 다시 라벨값으로 변환
        return output, h

encoder = Encoder(embedding_tensor).to(device)
decoder = Decoder(embedding_tensor).to(device)
loss_function = nn.CrossEntropyLoss()
encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr = 0.02)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr = 0.02)
epoch = 100
prev_acc = 0
acc_cnt = 0

for e in range(epoch) :
    encoder.train()
    decoder.train() #드롭아웃 활성화
    loss_sum = 0
    for x, t in train_dataloader :
# y = F(x)
        encoder_output, encoder_h = encoder(x)
        y, h, _ = decoder(encoder_output, encoder_h, t)
# y, t 비교
        loss = loss_function(y.reshape(-1, y.shape[-1]), t.reshape(-1)) #cross entropy loss 계산 위해서 3차원을 2차원으로 
        loss_sum += loss.item()
# F(x) 수정
        loss.backward()
        decoder_optimizer.step()
        encoder_optimizer.step()
        decoder_optimizer.zero_grad()
        encoder_optimizer.zero_grad()
    loss_sum /= len(train_dataloader)
# 중간 acc 점검, earlystopper 구현
    encoder.eval()
    decoder.eval() #드롭아웃 꺼주기
    correct = 0
    total = 0
    for x, t in test_dataloader :
        encoder_output, encoder_h = encoder(x)
        y, h, _ = decoder(encoder_output, encoder_h, None) #성능 평가기 때문에 t 입력 안함
        # for i in range(len(x)) :
        #     for j in range(4) :
        #         if (y[i][j].argmax() == t[i][j]) : #단어 하나하나씩 비교
        #             correct += 1
        correct += (y.argmax(dim = -1) == t).sum().item()
        total += len(x) * 4
    acc = correct / total

    if acc <= prev_acc :
        acc_cnt += 1
    else :
        acc_cnt = 0
        prev_acc = acc
    print(f"epoch {e+1} | loss {loss_sum} | acc {acc} | cnt {acc_cnt}")
    if acc_cnt >= 5:
        print("train stopped")
        break

epoch 1 | loss 1.564534757733345 | acc 0.468075 | cnt 0
epoch 2 | loss 1.143991583585739 | acc 0.534325 | cnt 0
epoch 3 | loss 1.024682889431715 | acc 0.574875 | cnt 0
epoch 4 | loss 0.9478731869161129 | acc 0.571775 | cnt 1
epoch 5 | loss 0.8979005862772464 | acc 0.6111 | cnt 0
epoch 6 | loss 0.8540154963731765 | acc 0.60755 | cnt 1
epoch 7 | loss 0.8161561946570873 | acc 0.62035 | cnt 0
epoch 8 | loss 0.7720776091516018 | acc 0.65405 | cnt 0
epoch 9 | loss 0.6958055786788464 | acc 0.68155 | cnt 0
epoch 10 | loss 0.6158981820940972 | acc 0.7133 | cnt 0
epoch 11 | loss 0.548891538232565 | acc 0.744 | cnt 0
epoch 12 | loss 0.5048222616314888 | acc 0.7627 | cnt 0
epoch 13 | loss 0.46004723213613036 | acc 0.769575 | cnt 0
epoch 14 | loss 0.4306331096589565 | acc 0.798475 | cnt 0
epoch 15 | loss 0.3958446746319532 | acc 0.7966 | cnt 1
epoch 16 | loss 0.36850445091724393 | acc 0.8129 | cnt 0
epoch 17 | loss 0.3467323074489832 | acc 0.8191 | cnt 0
epoch 18 | loss 0.3302297282963991 | acc 0.8

In [11]:
#신경망 저장

torch.save(encoder.to("cpu"), "num_encoder.pt")
torch.save(decoder.to("cpu"), "num_decoder.pt")